# d-separation

`d-separation` stands for `direction-separation` and it is a `rule` for reading off conditional independence relationships in a DAG. 


In [1]:
import networkx as nx

def get_dag(nodes, edges):
    g = nx.DiGraph()
    
    _ = [g.add_node(n) for n in nodes]
    _ = [g.add_edge(p, c) for p, c in edges]
    
    return g

nodes = ['D', 'I', 'G', 'S', 'L']
edges = [('D', 'G'), ('I', 'G'), ('I', 'S'), ('G', 'L')]

g = get_dag(nodes, edges)

In [2]:
g.nodes()

NodeView(('D', 'I', 'G', 'S', 'L'))

In [3]:
g.edges()

OutEdgeView([('D', 'G'), ('I', 'G'), ('I', 'S'), ('G', 'L')])

In [19]:
from networkx.algorithms.dag import ancestors, descendants
from networkx.algorithms.simple_paths import all_simple_paths

def get_paths(g, source, target):
    return all_simple_paths(g.to_undirected(), source, target)

def get_descendants(g):
    return {n: list(descendants(g, n)) for n in g.nodes()}

def is_path_active(g, path, Z, descendants):
    def is_collider(x, z, y):
        if g.has_edge(x, z) and g.has_edge(y, z):
            return True
        return False
    
    def is_collider_active(z):
        if z in Z:
            return True
        if len(set(descendants[z]) & set(Z)) > 0:
            return True
        return False
    
    def is_noncollider_active(z):
        if z in Z:
            return False
        return True
    
    for i, node in enumerate(path):
        if i > 0 and i < len(path) - 1:
            if is_collider(path[i-1], path[i], path[i+1]):
                is_active = is_collider_active(node)                
            else:
                is_active = is_noncollider_active(node)
            
            if not is_active:
                return False
                
    return True
            
is_path_active(g, ['D', 'G', 'I', 'S'], ['L', 'I'], get_descendants(g))

False

In [10]:
_, *rest, _ = ['D', 'G', 'I', 'S']

In [11]:
rest

['G', 'I']